## Age Phase 3 (Data Preparation): Title

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 12](https://data-science.training/kapitel-12/)
* Datum: 23.03.2023

Wir betrachten das nominalskalierte Attribut Name, genauer gesagt einen Teil davon, nämlich das Feature Title und führen hierzu das One Hote Encoding durch, um Dummy-Variablen zu bekommen. Diese Features analysieren wir dann in Bezug auf lineare Korrelationen zum Attribut Age.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
# NumPy Paket (Package) importieren
#  Mehrdimensionale Datenstrukturen (Vektoren, Matrizen, Tensoren, Arrays), Lineare Algebra
#  https://numpy.org/doc/
import numpy as np

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
df = pd.concat([df_train, df_test], ignore_index=True)

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Fehlende Werte behandeln (d.h. schätzen)
#  (KNIME: "Missing Values")
# Fare (Kardinalskala): 1 fehlender Wert => Benutze den konstanten Wert 7,896
#df['Fare'] = df['Fare'].fillna(7.896)
display(df[df['Fare'].isna()])
df.loc[1043, 'Fare'] = 7.896
display(df[df['PassengerId'] == 1044])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,<NA>,<NA>,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,7.896,<NA>,S


In [9]:
# Neues Feature Title
#  (KNIME: "Cell Splitter", "Column Rename", "Table Creator", "Cell Replacer")
df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
df['Title'] = df['Title'].replace(['Ms', 'Mlle'], 'Miss')
df['Title'] = df['Title'].replace(['Mme', 'Lady', 'the Countess', 'Dona'], 'Mrs')
df['Title'] = df['Title'].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'], 'Rare')

In [10]:
# One Hot Encoding => Dummy-Variablen
#  (KNIME: "One To Many")
df = pd.get_dummies(df, columns=['Title'], dtype=int)
# Ergebnis anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   1309 non-null   Int64  
 1   Survived      891 non-null    Int64  
 2   Pclass        1309 non-null   Int64  
 3   Name          1309 non-null   string 
 4   Sex           1309 non-null   string 
 5   Age           1046 non-null   Float64
 6   SibSp         1309 non-null   Int64  
 7   Parch         1309 non-null   Int64  
 8   Ticket        1309 non-null   string 
 9   Fare          1309 non-null   Float64
 10  Cabin         295 non-null    string 
 11  Embarked      1307 non-null   string 
 12  Title_Master  1309 non-null   int32  
 13  Title_Miss    1309 non-null   int32  
 14  Title_Mr      1309 non-null   int32  
 15  Title_Mrs     1309 non-null   int32  
 16  Title_Rare    1309 non-null   int32  
dtypes: Float64(2), Int64(5), int32(5), string(5)
memory usage: 157.4 KB


In [11]:
# Korrelationsmatrix mit den linearen Korrelationskoeffizienten nach Pearson
#  (KNIME: "Linear Correlation")
def dst_correlation_matrix(df):
    # Nur numerische Attribute auswählen
    df1 = df.select_dtypes(include=[np.number])
    # Korrelationsmatrix berechnen
    corr_matrix = df1.corr(method='pearson')
    # Rückgabe
    return corr_matrix

In [12]:
# Korrelationsmatrix berechnen und anzeigen
corr_matrix = dst_correlation_matrix(df)['Age'].sort_values()
display(corr_matrix)

Pclass         -0.408106
Title_Master   -0.391259
Title_Miss     -0.282792
SibSp          -0.243699
Parch          -0.150917
Survived       -0.077221
PassengerId     0.028814
Title_Rare      0.172905
Fare            0.177280
Title_Mr        0.183965
Title_Mrs       0.219841
Age             1.000000
Name: Age, dtype: float64

### Ergebnisse der Korrelationsanalyse der neuen Features zum Attribut Age

| Attribut           | R      |
|--------------------|--------|
| Title_Mrs          | +0,220 |
| Title_Mr           | +0,184 |
| Title_Rare         | +0,173 |
| Title_Miss         | -0,283 |
| Title_Master       | -0,391 |

Vereinfacht gesagt: Passagiere mit Titel Master oder Miss sind eher jung, während die anderen Passagiere eher alt sind.